In [1]:
import os
import random

import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, use_memory_fs=True, nb_workers=12)
import numpy as np

from tqdm.auto import tqdm
tqdm.pandas()

from utils.metrics import f1_macro

data_path = "./data/"
save_path = "./submissions/"

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
def decode(decoder_dict, key):
    for keys, value in decoder_dict.items():
        if key in keys:
            return value
    return key

        
def parse(address):
    pred = {"address": address}
    
    address = address.split(';')
    
    if address[0] != '  ':
        decoder_dict = {
            ("А.обл.", "Аобл", "аобл", "а.обл.",): "автономная область",
            ("А.окр.", "аокр", "а.окр.", "Аокр",): "автономный округ",
            ("Г", "Г.", "г", "г.", "город",): "город",
            ("край.", "Край.", "Край", "край",): "край",
            ("Обл", "Обл.", "обл", "обл.", "область",): "область",
            ("рес.", "рес", "Респ.", "Респ", "респ.", "респ", "Рес", "Рес.", "республика",): "республика",
        }
        
        address[0] = address[0].strip(' ')
        
        pred["region"] = ' '.join(address[0].split()[1:])
        pred["region_type"] = decode(decoder_dict, address[0].split()[0])
    
    if address[1] != '  ':
        decoder_dict = {
            ("р-н", "р-н.", "район",): "район",
            ("Г", "Г.", "г", "г.", "город",): "город",
            ("сп", "с/п", "сп.", "с.п.", "с/п.",): "сельское поселение",
            ("у", "у.", "У", "У.", "улус",): "улус",
            ("с/с", "сс", "сс.", "с/с.", "сельсовет",): "сельсовет",
            ("пгт", "пгт.",): "поселок городского типа",
            ("пос-к", "поселок",): "поселок",
            ("с", "с.", "село",): "село",
            ("с/мо", "смо.", "смо", "с/мо.",): "сельское муницип.образование",
            ("рп", "рп.",): "рабочий поселок",
            ("тер-я", "терр", "терр.", "тер.", "тер", "территория",): "территория",
            ("м-в", "м.", "м", "М.", "М", "массив",): "массив",
            ("пос-е", "поселение", "п", "п.", "пос", "пос.",): "поселение",
            ("зона",): "зона",
        }
        
        address[1] = address[1].strip(' ')
        
        pred["municipality"] = ' '.join(address[1].split()[1:])
        pred["municipality_type"] = decode(decoder_dict, address[1].split()[0])
        
    if address[2] != '  ':
        decoder_dict = {
            ("Г", "Г.", "г", "г.", "город",): "город",
            ("пос-к", "поселок", "п", "п.", "пос", "пос.",): "поселок",
            ("пгт.", "пгт",): "поселок городского типа",
            ("с", "с.", "село",): "село",
            ("ст", "ст.", "станция",): "станция",
            ("Д.", "д.", "д-ня", "д", "Д", "деревня",): "деревня",
            ("рп.", "рп",): "рабочий поселок",
            ("м-ние", "м-ние.", "месторождение",): "месторождение",
            ("сп", "с/п", "сп.", "с.п.", "с/п.",): "сельское поселение",
            ("х.", "х", "хутор",): "хутор",
            ("ст-ца", "ст-ца.", "станица",): "станица",
            ("к.п.", "кп.", "кп",): "курортный поселок",
            ("аул.", "аул",): "аул",
            ("м-в", "зона", "массив", "м", "м.", "М", "М.",): "массив",
            ("з-ка", "з-ка.", "заимка",): "заимка",
            ("у", "у.", "У", "У.", "улус",): "улус",
            ("в-ки", "в-ки.", "высел.", "высел", "выселки",): "выселки",
            ("рзд", "рзд.", "разъезд",): "разъезд",
            ("п/ст.", "п/ст", "п.ст.", "п.ст",): "поселок и(при) станция(и)",
            ("поч.", "поч", "п-к", "п-к.", "починок",): "починок",
            ("снт", "снт.",): "садовое неком-е товарищество",
            ("арбан.", "арбан",): "арбан",
            ("м-ко.", "м-ко", "местечко",): "местечко",
            ("нп", "нп.",): "населенный пункт",
            ("аал.", "аал",): "аал",
            ("тер-я", "терр", "терр.", "тер.", "тер", "территория",): "территория",
            ("о-в", "остров",): "остров",
            ("мкр", "мкр.", "микрорайон",): "микрорайон",
            ("с/с", "сс", "сс.", "с/с.", "сельсовет",): "сельсовет",
            ("г-к.", "г-к", "городок",): "городок",
            ("дп", "дп.", "д/п", "д/п.",): "дачный поселок",
            ("промзона",): "промышленная зона",
            ("кв-л.", "кв-л", "квартал",): "квартал",
            ("р-н", "р-н.", "район",): "район",
            ("к-ма.", "к-ма", "казарма",): "казарма",
            ("г/п", "гп", "гп.", "г/п.",): "городской поселок",
            ("с/а", "с/а.",): "сельская администрация",
            ("а/д", "а/д.", "ад", "ад.", "а.д.", "автодорога",): "автодорога",
            ("с/мо", "смо.", "смо", "с/мо.",): "сельское муницип.образование",
            ("сл", "сл.", "слобода",): "слобода",
            ("п/о", "п/о.",): "почтовое отделение",
            ("ж/р.", "ж/р", "ж.р.", "жилрайон", "ж.",): "жилой район",
            ("п/р", "п/р.",): "планировочный район",
            ("пос.рзд", "п.рзд"): "поселок разъезда",
            ("выселки(ок)",): "выселки(ок)",
            ("пос-е", "поселение",): "поселение",
            ("кордон",): "кордон",
            ("погост",): "погост",
        }
        
        address[2] = address[2].strip(' ')

        if address[2].split()[:2] == ["ж/д", "бл-ст."] or address[2].split()[:2] == ["ж/д", "бл-ст"]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "железнодорожный блокпост"
        elif address[2].split()[:2] == ["почтовое", "отделение"]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "почтовое отделение"
        elif address[2].split()[:2] == ["промышленная", "зона"]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "промышленная зона"
        elif address[2].split()[:2] == ["сельский", "поселок"]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "сельский поселок"
        elif address[2].split()[:2] == ["ж/д", "п.п."]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "железнодорожный путевой пост"
        elif address[2].split()[:2] == ["ж/д", "ст"] or address[2].split()[:2] == ["ж/д", "станция"] or address[2].split()[:2] == ["жд", "станция"] or address[2].split()[:2] == ["ж/д", "ст."]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "железнодорожная станция"
        elif address[2].split()[:2] == ["ж/д", "рзд"] or address[2].split()[:2] == ["ж/д", "р-д"] or address[2].split()[:2] == ["ж/д", "рзд."] or address[2].split()[:2] == ["ж/д", "р-д."]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "железнодорожный разъезд"
        elif address[2].split()[:2] == ["ж/д", "казарм"] or address[2].split()[:2] == ["ж/д", "казарм."] or address[2].split()[:2] == ["ж/д", "к-ма."] or address[2].split()[:2] == ["ж/д", "к-ма"]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "железнодорожная казарма"
        elif address[2].split()[:2] == ["ж/д", "оп."] or address[2].split()[:2] == ["ж/д", "о.п."] or address[2].split()[:2] == ["ж/д", "оп"]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "ж/д останов. (обгонный) пункт"
        elif address[2].split()[:2] == ["ж/д", "платф"] or address[2].split()[:2] == ["ж/д", "платф."] or address[2].split()[:2] == ["ж/д", "пл-ма."] or address[2].split()[:2] == ["ж/д", "пл-ма"]:
            pred["settlement"] = ' '.join(address[2].split()[2:])
            pred["settlement_type"] = "железнодорожная платформа"
        elif address[2].split()[:3] == ["п.", "ж/д", "ст."] or address[2].split()[:3] == ["п.", "ж/д", "ст"] or address[2].split()[:3] == ["п", "ж/д", "ст."] or address[2].split()[:3] == ["п", "ж/д", "ст"]:
            pred["settlement"] = ' '.join(address[2].split()[3:])
            pred["settlement_type"] = "поселок при железнодорожной станции"
        elif address[2].split()[:2] == ["ж/д", "б-ка"] or address[2].split()[:2] == ["ж/д", "б-ка."] or address[2].split()[:2] == ["ж/д", "будка"]:
            pred["settlement"] = ' '.join(address[3].split()[2:])
            pred["settlement_type"] = "железнодорожная будка"
        elif address[2].split()[:2] == ["ж/д", "пл-ка"] or address[2].split()[:2] == ["ж/д", "пл-ка."]:
            pred["settlement"] = ' '.join(address[3].split()[2:])
            pred["settlement_type"] = "железнодорожная площадка"
        elif address[2].split()[:3] == ["железнодорожный", "остановочный", "пункт"]:
            pred["settlement"] = ' '.join(address[3].split()[3:])
            pred["settlement_type"] = "железнодорожный остановочный пункт"
        else:
            pred["settlement"] = ' '.join(address[2].split()[1:])
            pred["settlement_type"] = decode(decoder_dict, address[2].split()[0])
        
    if address[3] != '  ':
        decoder_dict = {
            ("промзона",): "промышленная зона",
            ("снт", "снт.",): "садовое неком-е товарищество",
            ("тер-я", "терр", "терр.", "тер.", "тер", "территория",): "территория",
            ("рзд", "рзд.", "разъезд",): "разъезд",
            ("кв-л", "кв-л.", "квартал",): "квартал",
            ("ГКООП", "гкооп",): "гаражный кооператив",
            ("пос-к", "поселок", "п", "п.", "пос", "пос.",): "поселок",
            ("пос-е", "поселение",): "поселение",
            ("нп", "нп.",): "населенный пункт",
            ("пгт", "пгт.",): "поселок городского типа",
            ("мкр", "мкр.", "микрорайон",): "микрорайон",
            ("р-н", "р-н.", "район",): "район",
            ("о-в", "остров",): "остров",
            ("уч.", "уч", "участок",): "участок",
            ("ст", "ст.", "станция",): "станция",
            ("п/р",): "промышленный район",
            ("п.ст", "п.ст.",): "поселок при станции (поселок станции)",
            ("с", "с.", "село",): "село",
            ("платф", "платф.", "платформа",): "платформа",
            ("Д.", "д.", "д-ня", "д", "Д", "деревня",): "деревня",
            ("рп.", "рп",): "рабочий поселок",
            ("дп", "дп.", "д/п", "д/п.",): "дачный поселок",
            ("г/п", "гп", "гп.", "г/п.",): "городской поселок",
            ("х.", "х", "хутор",): "хутор",
            ("пл-ка", "пл-ка.", "площадка",): "площадка",
            ("ст-ца", "ст-ца.", "станица",): "станица",
            ("к-ма.", "к-ма","казарма",): "казарма",
            ("аул.", "аул",): "аул",
            ("ж/р.", "ж/р", "ж.р.", "жилрайон",): "жилой район",
            ("с/с", "сс", "сс.", "с/с.", "сельсовет",): "сельсовет",
            ("а/я",  "а/я.",): "абонентский ящик",
            ("м-ко.", "м-ко", "местечко",): "местечко",
            ("м", "м.", "М", "М.", "м-в", "м-в.", "зона", "массив",): "массив",
            ("п/о", "п/о.",): "почтовое отделение",
            ("сп", "с/п", "сп.", "с.п.", "с/п."): "сельское поселение",
            ("сл", "сл.", "слобода",): "слобода",
            ("к.п.", "кп.", "кп",): "курортный поселок",
            ("гск", "гск.",): "гаражно-строительный кооп.",
            ("м-ние", "м-ние.", "месторождение",): "месторождение",
            ("днп", "днп.",): "дачное неком-е партнерство",
            ("ф/х", "ф/х.",): "фермерское хозяйство",
            ("наб", "наб.", "набережная",): "набережная",
            ("ус", "усадьба",): "усадьба",
            ("ул", "ул.", "улица",): "улица",
            ("дор", "дор.", "дорога",): "дорога",
            ("км", "км.", "километр",): "километр",
            ("пер.", "пер-к", "пер", "пер-к.", "переулок",): "переулок",
            ("сад",): "сад",
            ("местность",): "местность",
            ("парк",): "парк",
            ("проезд",): "проезд",
            ("сквер",): "сквер",
            ("берег",): "берег",
            ("порт",): "порт",
            ("вал",): "вал",
        }
        
        address[3] = address[3].strip(' ')
        
        if address[3].split()[0] == "нп.":
            pred["location"] = ' '.join(address[3].split()[1:])
            pred["location_type"] = "населенный пункт"
        elif address[3].split()[:2] == ["промышленная", "зона"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "промышленная зона"
        elif address[3].split()[:3] == ["дачный", "населенный", "пункт"]:
            pred["location"] = ' '.join(address[3].split()[3:])
            pred["location_type"] = "дачный населенный пункт"
        elif address[3].split()[:2] == ["ж/д", "ст"] or address[3].split()[:2] == ["ж/д", "станция"] or address[3].split()[:2] == ["жд", "станция"] or address[3].split()[:2] == ["ж/д", "ст."]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "железнодорожная станция"
        elif address[3].split()[:2] == ["ж.", "р-н"] or address[3].split()[:2] == ["ж.", "р-н."]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "жилой район"
        elif address[3].split()[:2] == ["ж/д", "рзд"] or address[3].split()[:2] == ["ж/д", "р-д"] or address[3].split()[:2] == ["ж/д", "рзд."] or address[3].split()[:2] == ["ж/д", "р-д."]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "железнодорожный разъезд"
        elif address[3].split()[:2] == ["ж/д", "б-ка"] or address[3].split()[:2] == ["ж/д", "б-ка."] or address[3].split()[:2] == ["ж/д", "будка"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "железнодорожная будка"
        elif address[3].split()[:2] == ["почтовое", "отделение"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "почтовое отделение"
        elif address[3].split()[:2] == ["территория", "снт"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория снт"
        elif address[3].split()[:2] == ["территория", "дпк"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория дпк"
        elif address[3].split()[:2] == ["территория", "тсн"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория тсн"
        elif address[3].split()[:2] == ["территория", "днт"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория днт"
        elif address[3].split()[:2] == ["территория", "днп"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория днп"
        elif address[3].split()[:2] == ["территория", "спк"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория спк"
        elif address[3].split()[:2] == ["территория", "сно"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория сно"
        elif address[3].split()[:2] == ["территория", "дно"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория дно"
        elif address[3].split()[:2] == ["территория", "онт"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория онт"
        elif address[3].split()[:2] == ["территория", "оно"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория оно"
        elif address[3].split()[:2] == ["территория", "пк"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория пк"
        elif address[3].split()[:2] == ["садовое", "товарищество"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "садовое товарищество"
        elif address[3].split()[:2] == ["территория", "гск"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория гск"
        elif address[3].split()[:2] == ["территория", "снп"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория снп"
        elif address[3].split()[:2] == ["территория", "фх"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория фх"
        elif address[3].split()[:2] == ["территория", "сосн"]:
            pred["location"] = ' '.join(address[3].split()[2:])
            pred["location_type"] = "территория сосн"
        else:
            pred["location"] = ' '.join(address[3].split()[1:])
            pred["location_type"] = decode(decoder_dict, address[3].split()[0])
        
    if address[4] != '  ':
        decoder_dict = {
            ("ул", "ул.", "улица",): "улица",
            ("мкр", "мкр.", "микрорайон",): "микрорайон",
            ("пер.", "пер-к", "пер", "пер-к.", "переулок",): "переулок",
            ("пр-кт.", "пр-кт", "проспект",): "проспект",
            ("км", "км.", "километр",): "километр",
            ("пер-д", "переезд",): "переезд",
            ("пр-д", "проезд",): "проезд",
            ("ш", "ш.", "шоссе",): "шоссе",
            ("ГКООП", "гкооп",): "гаражный кооператив",
            ("дор", "дор.", "дорога",): "дорога",
            ("б-р", "бульвар",): "бульвар",
            ("ряд", "ряды", "ряд(ы)",): "ряд(ы)",
            ("наб", "наб.", "набережная",): "набережная",
            ("лн", "лн.", "линия",): "линия",
            ("пл.", "пл", "площадь",): "площадь",
            ("ф/х", "ф/х.",): "фермерское хозяйство",
            ("ф", "ф.", "ферма",): "ферма",
            ("туп", "туп.", "тупик",): "тупик",
            ("пр-ка", "просека",): "просека",
            ("кв-л.", "кв-л", "квартал",): "квартал",
            ("проул.", "проул", "проулок",): "проулок",
            ("мгстр", "мгстр.", "магистраль",): "магистраль",
            ("пр-лок", "проселок",): "проселок",
            ("тер-я", "терр", "терр.", "тер.", "тер", "территория",): "территория",
            ("снт", "снт.",): "садовое неком-е товарищество",
            ("гск", "гск.",): "гаражно-строительный кооп.",
            ("уч.", "уч", "участок",): "участок",
            ("рзд", "рзд.", "разъезд",): "разъезд",
            ("стр", "стр.", "строение",): "строение",
            ("х.", "х" "хутор",): "хутор",
            ("пос.", "пос-к", "пос", "поселок", "п", "п.",): "поселок",
            ("к-цо", "к-цо.", "кольцо",): "кольцо",
            ("ст", "ст.", "станция",): "станция",
            ("пл-ка", "пл-ка.", "площадка",): "площадка",
            ("г-к.", "г-к", "городок",): "городок",
            ("м-ко.", "м-ко", "м", "м.", "М", "М.", "местечко",): "местечко",
            ("сл", "сл.", "слобода",): "слобода",
            ("п/ст.", "п/ст", "п.ст.", "п.ст",): "поселок и(при) станция(и)",
            ("о-в", "остров",): "остров",
            ("жт", "жт.", "ж.т.",): "животноводческая точка",
            ("а/я",  "а/я.",): "абонентский ящик",
            ("платф", "платф.", "платформа",): "платформа",
            ("к-ма.", "к-ма", "казарма",): "казарма",
            ("нп", "нп.",): "населенный пункт",
            ("взв", "взв.", "взвоз",): "взвоз",
            ("Д.", "д.", "д-ня", "д", "Д", "деревня",): "деревня",
            ("промзона",): "промышленная зона",
            ("п/о", "п/о.",): "почтовое отделение",
            ("сад",): "сад",
            ("тракт",): "тракт",
            ("спуск",): "спуск",
            ("аллея",): "аллея",
            ("зона",): "зона",
            ("хутор", "х",): "хутор",
            ("парк",): "парк",
            ("вал",): "вал",
            ("коса",): "коса",
            ("заезд",): "заезд",
            ("въезд",): "въезд",
            ("сквер",): "сквер",
            ("село", "с.", "с",): "село",
            ("съезд", "сзд.", "сзд",): "съезд",
            ("местность",): "местность",
        }
        
        address[4] = address[4].strip(' ')

        if address[4].split()[:2] == ["ж/д", "б-ка"] or address[4].split()[:2] == ["ж/д", "б-ка."] or address[4].split()[:2] == ["ж/д", "будка"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "железнодорожная будка"
        elif address[4].split()[:2] == ["ж/д", "ст"] or address[4].split()[:2] == ["ж/д", "станция"] or address[4].split()[:2] == ["жд", "станция"] or address[4].split()[:2] == ["ж/д", "ст."]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "железнодорожная станция"
        elif address[4].split()[:2] == ["ж/д", "платф"] or address[4].split()[:2] == ["ж/д", "платф."] or address[4].split()[:2] == ["ж/д", "пл-ма."] or address[4].split()[:2] == ["ж/д", "пл-ма"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "железнодорожная платформа"
        elif address[4].split()[:2] == ["ж/д", "оп."] or address[4].split()[:2] == ["ж/д", "о.п."] or address[4].split()[:2] == ["ж/д", "оп"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "ж/д останов. (обгонный) пункт"
        elif address[4].split()[:2] == ["ж/д", "рзд"] or address[4].split()[:2] == ["ж/д", "рзд."]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "железнодорожный разъезд"
        elif address[4].split()[:2] == ["ж/д", "к-ма"] or address[4].split()[:2] == ["ж/д", "казарм."] or address[4].split()[:2] == ["ж/д", "казарм"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "железнодорожная казарма"
        elif address[4].split()[:2] == ["ж/д", "р-д"] or address[4].split()[:2] == ["ж/д", "р-д."]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "железнодорожный разъезд"
        elif address[4].split()[:2] == ["территория", "снт"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "территория снт"
        elif address[4].split()[:2] == ["промышленная", "зона"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "промышленная зона"
        elif address[4].split()[:2] == ["почтовое", "отделение"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "почтовое отделение"
        elif address[4].split()[:2] == ["садовое", "товарищество"]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "садовое товарищество"
        elif address[4].split()[:2] == ["железнодорожный", "пост"] or address[4].split()[:2] == ["ж/д", "пост"] or address[4].split()[:2] == ["ж/д", "п."]:
            pred["street"] = ' '.join(address[4].split()[2:])
            pred["street_type"] = "железнодорожный пост"
        else:
            pred["street"] = ' '.join(address[4].split()[1:])
            pred["street_type"] = decode(decoder_dict, address[4].split()[0])
            
    if address[5] != '  ':
        pred["house"] = address[5].strip(' ')
        
    if address[6] != '  ':
        pred["source"] = int(address[6].strip(' '))
        
#     for key in pred.keys():
#         pred[key] = [pred[key]]
    
    return pd.Series(pred)

In [3]:
df_test = pd.read_csv(os.path.join(data_path, "addresses-close.csv"), encoding='windows-1251', dtype=str)
df_test.shape

(5112420, 1)

In [4]:
df_test = df_test["address"].parallel_apply(parse)
df_test = df_test.fillna('')

/home/egor/PycharmProjects/rucode_7/venv/lib/python3.11/site-packages/pandarallel/data_types/series.py:26: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  return data.apply(
/home/egor/PycharmProjects/rucode_7/venv/lib/python3.11/site-packages/pandarallel/data_types/series.py:26: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  return data.apply(
/home/egor/PycharmProjects/rucode_7/venv/lib/python3.11/site-packages/pandarallel/data_types/series.py:26: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  return data.apply(
/home/egor/PycharmProjects/rucode_7/venv/lib/python3.11/site-packages/pandarallel/data_types/series.py:26: FutureWarning: Returning a DataFrame from Serie

In [5]:
df_test = df_test.drop(columns=["address"])

In [6]:
df_test

,region,region_type,settlement,settlement_type,street,street_type,house,source,location,location_type,municipality,municipality_type
0,Еврейская,автономная область,Биробиджан,город,Авангардный,переулок,дом 6,1,,,,
1,Еврейская,автономная область,Биробиджан,город,Алмазный,переулок,дом 11,2,,,,
2,Еврейская,автономная область,Биробиджан,город,Ангарский,переулок,дом 2,2,,,,
3,Еврейская,автономная область,Биробиджан,город,Апрельский,переулок,дом 1Б,2,,,,
4,Еврейская,автономная область,Биробиджан,город,Апрельский,переулок,дом 6,2,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
5112415,Севастополь,город,,,,,дом 129,1,ТСН СТ Автомобилист,территория тсн,,
5112416,Севастополь,город,,,5-я Узорная,улица,дом 9,1,ТСН СТ Овощевод,территория тсн,,
5112417,Севастополь,город,,,,,дом 22,1,ТСН ДНТ Волга,территория,,
5112418,Севастополь,город,,,,,дом 21,1,Коллективный сад,территория тсн,,


In [7]:
df_test.to_csv(os.path.join(save_path, "close_1.csv"), index=False, encoding="windows-1251")